In [1]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import numpy as np
from random import random, seed, randrange
%matplotlib inline
seed=1

def FrankeFunction(x,y):
    term1 = 0.75*np.exp(-(0.25*(9*x-2)**2) - 0.25*((9*y-2)**2))
    term2 = 0.75*np.exp(-((9*x+1)**2)/49.0 - 0.1*(9*y+1))
    term3 = 0.5*np.exp(-(9*x-7)**2/4.0 - 0.25*((9*y-3)**2))
    term4 = -0.2*np.exp(-(9*x-4)**2 - (9*y-7)**2)
    return term1 + term2 + term3 + term4


def MSE(y, yh):
     return np.square(y - yh).mean()
def r2score(y,yh):    
    return (1 - np.sum(np.square(y-yh))/np.sum(np.square(y- np.mean(yh))))
def cofidentint(XXinv,y,yh,beta):
    
    N=np.size(y,0)

    p=np.size(XXinv,0)

    confInt=np.zeros((p,2))
    sigma2=np.sum(np.square(y-yh))/(N-p-1)

    for i in range(p):
        confInt[i,0]=beta[i]-1.645*np.sqrt(abs(XXinv[i,i]))*sigma2
        confInt[i,1]=beta[i]+1.645*np.sqrt(abs(XXinv[i,i]))*sigma2
    return confInt
    
def OSLregression(xVector,yVector,zVector,polyOrder):
    
    vectorSize=np.size(yVector,0)
    #transform back to a matrix
    if polyOrder==3:       
        xMatrix = np.c_[np.ones((vectorSize,1)), xVector, yVector,xVector**2,yVector**2,xVector*yVector,
                   xVector**3,yVector**3,(xVector**2)*yVector,(yVector**2)*xVector]
    elif polyOrder==4:
        xMatrix= np.c_[np.ones((vectorSize,1)), xVector, yVector,xVector**2,yVector**2,xVector*yVector,
                   xVector**3,yVector**3,(xVector**2)*yVector,(yVector**2)*xVector,
                      xVector**4,xVector*(yVector**3),(xVector**3)*yVector,(xVector**2)*(yVector**2),
                       (xVector**3)*yVector,yVector**4,
                       (xVector**2)*(yVector**2),xVector*(yVector**3)]
    elif polyOrder==5:
        xMatrix=np.c_[np.ones((vectorSize,1)), xVector, yVector,xVector**2,yVector**2,xVector*yVector,
                   xVector**3,yVector**3,(xVector**2)*yVector,(yVector**2)*xVector,
                      xVector**4,xVector*(yVector**3),(xVector**3)*yVector,(xVector**2)*(yVector**2),
                       (xVector**3)*yVector,yVector**4,
                       (xVector**2)*(yVector**2),xVector*(yVector**3),
                      xVector**5,(xVector**2)*(yVector**3),(xVector**4)*yVector,(xVector**3)*(yVector**2),
                       (xVector**4)*yVector,xVector*(yVector**4),
                       (xVector**3)*(yVector**2),(xVector**2)*(yVector**3),
                     xVector**4*yVector,xVector*(yVector**4),(xVector**3)*(yVector**2),(xVector**2)*(yVector**3),
                       (xVector**3)*(yVector**2),yVector**5,
                       (xVector**2)*(yVector**3),xVector*(yVector**4)]
    #pseudo inversion using SVD   
    XXinv=np.linalg.pinv(xMatrix.T.dot(xMatrix))
    beta = XXinv.dot(xMatrix.T).dot(zVector)
    #print(beta)
    #zPredict=xMatrix.dot(beta)
    #zPredictReshape=np.reshape(zPredict,(10,10))
    return beta,XXinv

def Ridgeregression(xVector,yVector,zVector,polyOrder,lambda1):
    
    vectorSize=np.size(yVector,0)

    #transform back to a matrix
    #
    if polyOrder==3:       
        xMatrix = np.c_[np.ones((vectorSize,1)), xVector, yVector,xVector**2,yVector**2,xVector*yVector,
                   xVector**3,yVector**3,(xVector**2)*yVector,(yVector**2)*xVector]
    elif polyOrder==4:
        xMatrix= np.c_[np.ones((vectorSize,1)), xVector, yVector,xVector**2,yVector**2,xVector*yVector,
                   xVector**3,yVector**3,(xVector**2)*yVector,(yVector**2)*xVector,
                      xVector**4,xVector*(yVector**3),(xVector**3)*yVector,(xVector**2)*(yVector**2),
                       (xVector**3)*yVector,yVector**4,
                       (xVector**2)*(yVector**2),xVector*(yVector**3)]
    elif polyOrder==5:
        xMatrix=np.c_[np.ones((vectorSize,1)), xVector, yVector,xVector**2,yVector**2,xVector*yVector,
                   xVector**3,yVector**3,(xVector**2)*yVector,(yVector**2)*xVector,
                      xVector**4,xVector*(yVector**3),(xVector**3)*yVector,(xVector**2)*(yVector**2),
                       (xVector**3)*yVector,yVector**4,
                       (xVector**2)*(yVector**2),xVector*(yVector**3),
                      xVector**5,(xVector**2)*(yVector**3),(xVector**4)*yVector,(xVector**3)*(yVector**2),
                       (xVector**4)*yVector,xVector*(yVector**4),
                       (xVector**3)*(yVector**2),(xVector**2)*(yVector**3),
                     xVector**4*yVector,xVector*(yVector**4),(xVector**3)*(yVector**2),(xVector**2)*(yVector**3),
                       (xVector**3)*(yVector**2),yVector**5,
                       (xVector**2)*(yVector**3),xVector*(yVector**4)]
    size1=np.size(xMatrix,1)
    I=np.identity(size1)
    #pseudo inversion using SVD
    XXinv=np.linalg.pinv(xMatrix.T.dot(xMatrix)+ lambda1*I)
    beta = XXinv.dot(xMatrix.T).dot(zVector)
    #print(beta)
    #zPredict=xMatrix.dot(beta)
    #zPredictReshape=np.reshape(zPredict,(10,10))
    return beta,XXinv

def computeZpredict(xVector,yVector,beta,polyOrder):
    #
    vectorSize=np.size(yVector,0)
    
    if polyOrder==3:       
        xMatrix = np.c_[np.ones((vectorSize,1)), xVector, yVector,xVector**2,yVector**2,xVector*yVector,
                   xVector**3,yVector**3,(xVector**2)*yVector,(yVector**2)*xVector]
    elif polyOrder==4:
        xMatrix= np.c_[np.ones((vectorSize,1)), xVector, yVector,xVector**2,yVector**2,xVector*yVector,
                   xVector**3,yVector**3,(xVector**2)*yVector,(yVector**2)*xVector,
                      xVector**4,xVector*(yVector**3),(xVector**3)*yVector,(xVector**2)*(yVector**2),
                       (xVector**3)*yVector,yVector**4,
                       (xVector**2)*(yVector**2),xVector*(yVector**3)]
    elif polyOrder==5:
        xMatrix=np.c_[np.ones((vectorSize,1)), xVector, yVector,xVector**2,yVector**2,xVector*yVector,
                   xVector**3,yVector**3,(xVector**2)*yVector,(yVector**2)*xVector,
                      xVector**4,xVector*(yVector**3),(xVector**3)*yVector,(xVector**2)*(yVector**2),
                       (xVector**3)*yVector,yVector**4,
                       (xVector**2)*(yVector**2),xVector*(yVector**3),
                      xVector**5,(xVector**2)*(yVector**3),(xVector**4)*yVector,(xVector**3)*(yVector**2),
                       (xVector**4)*yVector,xVector*(yVector**4),
                       (xVector**3)*(yVector**2),(xVector**2)*(yVector**3),
                     xVector**4*yVector,xVector*(yVector**4),(xVector**3)*(yVector**2),(xVector**2)*(yVector**3),
                       (xVector**3)*(yVector**2),yVector**5,
                       (xVector**2)*(yVector**3),xVector*(yVector**4)]
    zPredict=xMatrix.dot(beta)
    return zPredict

def plotTheSurface(x,y,z):
# Plot the surface.
    fig = plt.figure()
    ax = fig.gca(projection='3d')


    surf = ax.plot_surface(x, y, z, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
# Customize the z axis.
#ax.set_zlim(-0.10, 1.40)
#ax.zaxis.set_major_locator(LinearLocator(10))
#ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))
# Add a color bar which maps values to colors.
#fig.colorbar(surf, shrink=0.5, aspect=5)
    plt.show();

def train_test_split(dataset, split):
    # Split a dataset into a train and test set
    train = list()
    train_size = split * len(dataset)
    dataset_copy = list(dataset)
    while len(train) < train_size:
        index = randrange(len(dataset_copy))
        #print(index)
        train.append(dataset_copy.pop(index))
    #tranform into np.array
    train=np.array(train)
    dataset_copy=np.array(dataset_copy)
    return train, dataset_copy



    
def k_folds_CV(dataset, nfolds):
    # Split a dataset into k folds
    splitedDataset = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / nfolds)
    for i in range(nfolds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        splitedDataset.append(fold)
    return splitedDataset

def trainSetindex(indeces,testSetindex):
    #given indeces of the test set, find the indeces of the train set
    size=np.size(indeces)
    mask = np.ones(size, dtype=bool)
    mask[testSetindex] = False
    return indeces[mask]
def computeBiasandVar(zPredictmatrix,zVector):
    n=np.size(zPredictmatrix,0)
    m=np.size(zPredictmatrix,1)
    meanzPredictmatrix=np.mean(zPredictmatrix,1)
    bias=np.sum(np.square(zVector-meanzPredictmatrix))
    newMatrix=np.zeros((n,m))
    for i in range(m-1):
        newMatrix[:,i]=zPredictmatrix[:,i]-meanzPredictmatrix
    newMatrix=np.square(newMatrix)
    newMatrix=np.mean(newMatrix)
    var=np.sum(newMatrix,0)
    return bias,var

def constructX(xVector,yVector,polyOrder):
    
    vectorSize=np.size(yVector,0)
    if polyOrder==3:       
        xMatrix = np.c_[np.ones((vectorSize,1)), xVector, yVector,xVector**2,yVector**2,xVector*yVector,
                   xVector**3,yVector**3,(xVector**2)*yVector,(yVector**2)*xVector]
    elif polyOrder==4:
        xMatrix= np.c_[np.ones((vectorSize,1)), xVector, yVector,xVector**2,yVector**2,xVector*yVector,
                   xVector**3,yVector**3,(xVector**2)*yVector,(yVector**2)*xVector,
                      xVector**4,xVector*(yVector**3),(xVector**3)*yVector,(xVector**2)*(yVector**2),
                       (xVector**3)*yVector,yVector**4,
                       (xVector**2)*(yVector**2),xVector*(yVector**3)]
    elif polyOrder==5:
        xMatrix=np.c_[np.ones((vectorSize,1)), xVector, yVector,xVector**2,yVector**2,xVector*yVector,
                   xVector**3,yVector**3,(xVector**2)*yVector,(yVector**2)*xVector,
                      xVector**4,xVector*(yVector**3),(xVector**3)*yVector,(xVector**2)*(yVector**2),
                       (xVector**3)*yVector,yVector**4,
                       (xVector**2)*(yVector**2),xVector*(yVector**3),
                      xVector**5,(xVector**2)*(yVector**3),(xVector**4)*yVector,(xVector**3)*(yVector**2),
                       (xVector**4)*yVector,xVector*(yVector**4),
                       (xVector**3)*(yVector**2),(xVector**2)*(yVector**3),
                     xVector**4*yVector,xVector*(yVector**4),(xVector**3)*(yVector**2),(xVector**2)*(yVector**3),
                       (xVector**3)*(yVector**2),yVector**5,
                       (xVector**2)*(yVector**3),xVector*(yVector**4)]

    return xMatrix

In [2]:
x = np.arange(0, 1, 0.01)
y = np.arange(0, 1, 0.01)
#x = np.random.rand(100,1)
#y = np.random.rand(100,1)
#print(x)
#print(y)
x, y = np.meshgrid(x,y)
z = FrankeFunction(x, y) +0.02*np.random.randn(100,100)
#plotTheSurface(x,y,z)

#OSL regression




In [3]:
#transforming to a coluum vector in order to form an x_hat matrix
xVector=np.matrix.flatten(x)
yVector=np.matrix.flatten(y)
#vectorSize=np.size(yVector,0)
zVector=np.matrix.flatten(z)
#print(np.size(y,1))
#transform back to a matrix
#yReshape=np.reshape(y,(100,100))
#beta,XXinv=OSLregression(xVector,yVector,zVector,3)
#zPredict=computeZpredict(xVector,yVector,beta,3)

#zPredictReshape=np.reshape(zPredict,(100,100))

In [ ]:
matrixSize=np.size(y,0)
zPredictReshape=np.reshape(zPredict,(matrixSize,matrixSize))
#plotTheSurface(x,y,zPredictReshape)

In [ ]:

print(MSE(zVector,zPredict))
print(r2score(zVector,zPredict))
print(cofidentint(XXinv,zVector,zPredict,beta))

In [ ]:
beta,XXinv=Ridgeregression(xVector,yVector,zVector,3,1)
zPredict=computeZpredict(xVector,yVector,beta,3)
print(MSE(zVector,zPredict))
print(r2score(zVector,zPredict))
print(cofidentint(XXinv,zVector,zPredict,beta))


In [ ]:
matrixSize=np.size(y,0)
zPredictReshape=np.reshape(zPredict,(matrixSize,matrixSize))
plotTheSurface(x,y,zPredictReshape)

In [ ]:

#dataset = [[1], [2], [3], [4], [5], [6], [7], [8], [9], [10]]
sizeVector=np.size(zVector)
train, test = train_test_split(np.linspace(0,sizeVector-1,sizeVector),0.5)
train=train.astype(int)
test=test.astype(int)
print(train)
print(test)

In [ ]:
#xVector[train]
#xVector[test]
beta,XXinv=Ridgeregression(xVector[train],yVector[train],zVector[train],3,0.1)
zPredict=computeZpredict(xVector[test],yVector[test],beta,3)
print(MSE(zVector[test],zPredict))
print(r2score(zVector[test],zPredict))
#print(cofidentint(XXinv,zVector[train],zPredict[train],beta))



In [4]:
sizeVector=np.size(zVector)
#print(sizeVector)

In [6]:


 
# test cross validation split

#dataset = [[1], [2], [3], [4], [5], [6], [7], [8], [9], [10]]
indeces=np.linspace(0,sizeVector-1,sizeVector)
indeces=indeces.astype(int)
numberOfFolds=10
folds = k_folds_CV(indeces, numberOfFolds)
#print(folds[0])
#test1=folds[0]
#train1= trainSetindex(indeces,test1)
#print(train1)

In [7]:
#OLS Model
polynom_oders=[3,4,5]
numOfoders=len(polynom_oders)
statsMatrix=np.zeros((2,numberOfFolds,numOfoders))
zPredictmatrix=np.zeros((sizeVector,numberOfFolds,numOfoders))
for j,order in enumerate(polynom_oders):
    for i in range(numberOfFolds):
        #print(i)
    
        test1=folds[i]
        train1= trainSetindex(indeces,test1)
        beta,XXinv=OSLregression(xVector[train1],yVector[train1],zVector[train1],order)
    #zPredict=computeZpredict(xVector[test1],yVector[test1],beta,3)
        zPredictmatrix[:,i,j]=computeZpredict(xVector,yVector,beta,order)
        statsMatrix[0,i,j]=MSE(zVector[test1],zPredictmatrix[test1,i,j])
        statsMatrix[1,i,j]=r2score(zVector[test1],zPredictmatrix[test1,i,j])
    print('Average of MSE for polynom order {} is:'.format(str(order)) + str(np.mean(statsMatrix[0,:,j])))
    print('Average of R2score for polynom order {} is:'.format(str(order)) + str(np.mean(statsMatrix[1,:,j])))
    print('\n')


Average of MSE for polynom order 3 is:0.008158656805974513
Average of R2score for polynom order 3 is:0.901532574897255


Average of MSE for polynom order 4 is:0.0043670560695081885
Average of R2score for polynom order 4 is:0.947262737808327


Average of MSE for polynom order 5 is:0.0024625477457780594
Average of R2score for polynom order 5 is:0.9702710677180166




In [ ]:
#final OLS model
# bias and var Only for the best model
bias,var=computeBiasandVar(zPredictmatrix[:,:,2],zVector)
print('Bias^2 for the final model is: ' + str(bias))
print('Var for the final model is: ' + str(var))
z_pred=np.mean(zPredictmatrix[:,:,2],1)
matrixSize=np.size(y,0)
zPredictReshape=np.reshape(z_pred,(matrixSize,matrixSize))
plotTheSurface(x,y,zPredictReshape)


In [ ]:
vectorSize=np.size(yVector,0)
xMatrix = np.c_[np.ones((vectorSize,1)), xVector, yVector,xVector**2,yVector**2,xVector*yVector,
                   xVector**3,yVector**3,(xVector**2)*yVector,(yVector**2)*xVector]
#U, s, VT = np.linalg.svd(xMatrix)
# create m x n Sigma matrix
#Sigma = np.zeros((xMatrix.shape[0], xMatrix.shape[1]))
# populate Sigma with n x n diagonal matrix
#Sigma[:xMatrix.shape[1], :xMatrix.shape[1]] = np.diag(s)
# reconstruct matrix
#B = U.dot(Sigma.dot(VT))


In [ ]:
zz=U.dot(np.transpose(U).dot(zVector))
print(zz-zVector)

In [ ]:
print(MSE(zVector,zPredict))
print(r2score(zVector,zPredict))
print(cofidentint(XXinv,zVector,zPredict,beta))

In [ ]:
print(MSE(zVector,zz))
print(r2score(zVector,zz))
print(cofidentint(XXinv,zVector,zz,beta))

In [ ]:
from sklearn.linear_model import Lasso
lassoreg = Lasso(alpha=0.0001,normalize=True)
lassoreg.fit(xMatrix,zVector)
z_pred = lassoreg.predict(xMatrix)

In [ ]:
print(MSE(zVector,z_pred))
print(r2score(zVector,z_pred))
#print(cofidentint(XXinv,zVector,zPredict,beta))

In [ ]:
from sklearn.linear_model import Ridge
ridgereg = Ridge(alpha=0,normalize=True)
ridgereg.fit(xMatrix,zVector)
z_pred = ridgereg.predict(xMatrix)
print(MSE(zVector,z_pred))
print(r2score(zVector,z_pred))

In [ ]:

#Ridge Model
lambda_values=[1e-3,1e-2,1,10,1e2,1e3]
numOfLambdas=len(lambda_values)
polynom_oders=[3,4,5]
numOfoders=len(polynom_oders)
statsMatrix=np.zeros((2,numberOfFolds,numOfLambdas,numOfoders))
zPredictmatrix=np.zeros((sizeVector,numberOfFolds,numOfLambdas,numOfoders))
for j,order in enumerate(polynom_oders):
    for i in range(numberOfFolds):
        #print(i)
        test1=folds[i]
        train1= trainSetindex(indeces,test1)
        for  h,lbd in enumerate(lambda_values):           
            beta,XXinv=Ridgeregression(xVector[train1],yVector[train1],zVector[train1],order,lbd)
    #zPredict=computeZpredict(xVector[test1],yVector[test1],beta,3)
            zPredictmatrix[:,i,h,j]=computeZpredict(xVector,yVector,beta,order)
            statsMatrix[0,i,h,j]=MSE(zVector[test1],zPredictmatrix[test1,i,h,j])
            statsMatrix[1,i,h,j]=r2score(zVector[test1],zPredictmatrix[test1,i,h,j])

       

In [ ]:
#printing results
for j,order in enumerate(polynom_oders):
    for  h,lbd in enumerate(lambda_values): 
        print(lbd)
        print('Average of MSE for polynom order {} is:'.format(str(order)) + str(np.mean(statsMatrix[0,:,h,j])))
        print('Average of R2score for polynom order {} is:'.format(str(order)) + str(np.mean(statsMatrix[1,:,h,j])))
        print('\n')
    

In [ ]:

bias,var=computeBiasandVar(zPredictmatrix[:,:,0,2],zVector)
print('Bias^2 for the final model is: ' + str(bias))
print('Var for the final model is: ' + str(var))
z_pred=np.mean(zPredictmatrix[:,:,0,2],1)
matrixSize=np.size(y,0)
zPredictReshape=np.reshape(z_pred,(matrixSize,matrixSize))
plotTheSurface(x,y,zPredictReshape)

In [8]:
XMatrix=constructX(xVector,yVector,3)

In [11]:
#Lasso Model
from sklearn.linear_model import Lasso 
from sklearn.metrics import mean_squared_error, r2_score

lambda_values=[1e-3,1e-2,1,10,1e2,1e3]
numOfLambdas=len(lambda_values)
polynom_oders=[3,4,5]
numOfoders=len(polynom_oders)
statsMatrix=np.zeros((2,numberOfFolds,numOfLambdas,numOfoders))
zPredictmatrix=np.zeros((sizeVector,numberOfFolds,numOfLambdas,numOfoders))

for j,order in enumerate(polynom_oders):
    XMatrix=constructX(xVector,yVector,order)
    for i in range(numberOfFolds):
        #print(i)
        test1=folds[i]
        train1= trainSetindex(indeces,test1)
        for  h,lbd in enumerate(lambda_values):           
            #beta,XXinv=Ridgeregression(xVector[train1],yVector[train1],zVector[train1],order,lbd)
    #zPredict=computeZpredict(xVector[test1],yVector[test1],beta,3)
            
            lasso=Lasso(lbd,max_iter=1000)
            lasso.fit(XMatrix[train1,:],zVector[train1])
            zPredictmatrix[:,i,h,j]=lasso.predict(XMatrix)
            #zPredictmatrix[:,i,h,j]=computeZpredict(xVector,yVector,beta,order)
            statsMatrix[0,i,h,j]=mean_squared_error(zVector[test1],zPredictmatrix[test1,i,h,j])
            statsMatrix[1,i,h,j]=r2_score(zVector[test1],zPredictmatrix[test1,i,h,j])



In [12]:
for j,order in enumerate(polynom_oders):
    for  h,lbd in enumerate(lambda_values): 
        print(lbd)
        print('Average of MSE for polynom order {} is:'.format(str(order)) + str(np.mean(statsMatrix[0,:,h,j])))
        print('Average of R2score for polynom order {} is:'.format(str(order)) + str(np.mean(statsMatrix[1,:,h,j])))
        print('\n')

0.001
Average of MSE for polynom order 3 is:0.018000672646189982
Average of R2score for polynom order 3 is:0.7827052945371255


0.01
Average of MSE for polynom order 3 is:0.025392171882509928
Average of R2score for polynom order 3 is:0.6937171925684784


1
Average of MSE for polynom order 3 is:0.08297233843368836
Average of R2score for polynom order 3 is:-0.0015164427569810846


10
Average of MSE for polynom order 3 is:0.08297233843368836
Average of R2score for polynom order 3 is:-0.0015164427569810846


100.0
Average of MSE for polynom order 3 is:0.08297233843368836
Average of R2score for polynom order 3 is:-0.0015164427569810846


1000.0
Average of MSE for polynom order 3 is:0.08297233843368836
Average of R2score for polynom order 3 is:-0.0015164427569810846


0.001
Average of MSE for polynom order 4 is:0.014252865577506824
Average of R2score for polynom order 4 is:0.8279798240186373


0.01
Average of MSE for polynom order 4 is:0.025392171882509928
Average of R2score for polynom orde